In [1]:
import bw2io as bi
import bw2data as bd
import sys
sys.path.append("/Users/akim/PycharmProjects/bigfood_db")

# Local files
from bigfood_db.import_databases import import_ecoinvent
from bigfood_db.importers import Agribalyse13Importer
from bigfood_db.utils import create_location_mapping, modify_exchanges

if __name__ == '__main__':
    project = "Food ei38"
    bd.projects.set_current(project)

    ei_path = "/Users/akim/Documents/LCA_files/ecoinvent_38_cutoff/datasets"
    ei_name = "ecoinvent 3.8 cutoff"

    bi.bw2setup()
    b3 = bd.Database('biosphere3')
    import_ecoinvent(ei_path, ei_name)
    ei = bd.Database(ei_name)

    ag_path = "/Users/akim/Documents/LCA_files/agribalyse_13/Agribalyse CSV FINAL_no links_Nov2016v3.CSV"
    ag_name = "Agribalyse 1.3"
    ag = Agribalyse13Importer(ag_path, ag_name)
#     ag.add_unlinked_activities()
#     mapping = create_location_mapping(ag, ei_name)
#     ag = modify_exchanges(ag, mapping, ei_name)
    

Biosphere database already present!!! No setup is needed
ecoinvent 3.8 cutoff database already present!!! No import is needed
Extracted 1189 unallocated datasets in 3.97 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: assign_only_product_as_production
Applying strategy: drop_unspecified_subcategories
Applying strategy: sp_allocate_products
Applying strategy: fix_zero_allocation_products
Applying strategy: split_simapro_name_geo
Applying strategy: strip_biosphere_exc_locations
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: set_code_by_activity_hash
Applying strategy: link_technosphere_based_on_name_unit_location
Applying strategy: change_electricity_unit_mj_to_kwh
Applying strategy: set_lognormal_loc_value_uncertainty_safe
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying s

In [2]:
ag.statistics()

1261 datasets
115259 exchanges
3294 unlinked exchanges
  Type biosphere: 440 unique unlinked exchanges
  Type technosphere: 77 unique unlinked exchanges


(1261, 115259, 3294)

In [ ]:
[act['name'] for act in ag.unlinked if "tech" in act['type']]

# Biosphere

In [ ]:
# del bd.databases["Agribalyse biosphere"]
# bd.Database("Agribalyse biosphere").register()
# ag.add_unlinked_flows_to_biosphere_database("Agribalyse biosphere")

# Technosphere

In [34]:
def create_location_mapping_glo(db1, db2_name):
    """
    Create mapping between unlinked exchanges of database `db1` and their corresponding activities in `db2`.
    Here an exchange in `db1` needs to be linked to activity in `db2` with location `GLO`, which does not exist there.
    Instead, there are activities with the same name but different locations, which constitute GLO location together.
    For example, CH + DE + RoW = GLO, where Rest of the World (RoW) basically adds whatever locations are missing to
    result in GLO. Hence, the mapping is from unlinked exchanges of `db1` and list of activities in `db2` with same name
    but multiple locations.
    Example: (market for lime, GLO) is split by production volume into
             (market for lime, RoW) & (market for lime, RER).
    Mapping is a list of dictionaries where each dictionary corresponds to an unlinked exchange.
    The key is the index of the unlinked exchange in `db`` and the values are `db2` activities codes.
    """
    db2 = bd.Database(db2_name)
    unlinked_list = [act for act in db1.unlinked if act['type'] == 'technosphere' and act.get('location') == "GLO"]
    mapping = []
    for i, activity in enumerate(unlinked_list):
        new_el = {}
        name = activity['name']
        loc = activity['location']
        acts_codes = [act for act in db2 if name == act['name']]
        if len(acts_codes)>0:
            new_el[(name, loc)] = acts_codes
            mapping.append(new_el)
    return mapping

In [35]:
def create_location_mapping_rer(db1, db2_name):
    """
    Create mapping between unlinked exchanges of database `db1` and their corresponding activities in `db2`.
    Here an exchange in `db1` needs to be linked to activity in `db2` with location `GLO`, which does not exist there.
    Instead, there are activities with the same name but different locations, which constitute GLO location together.
    For example, CH + DE + RoW = GLO, where Rest of the World (RoW) basically adds whatever locations are missing to
    result in GLO. Hence, the mapping is from unlinked exchanges of `db1` and list of activities in `db2` with same name
    but multiple locations.
    Example: (market for lime, GLO) is split by production volume into
             (market for lime, RoW) & (market for lime, RER).
    Mapping is a list of dictionaries where each dictionary corresponds to an unlinked exchange.
    The key is the index of the unlinked exchange in `db`` and the values are `db2` activities codes.
    """
    db2 = bd.Database(db2_name)
    unlinked_list = [act for act in db1.unlinked if act['type'] == 'technosphere' and act.get('location') == "RER"]
    mapping = []
    import country_converter as coco
    for i, activity in enumerate(unlinked_list):
        new_el = {}
        name = activity['name']
        loc = activity['location']
        for rer_wo_string in ["Europe without ", "RER w/o "]:
            act_rer_wo_xx = [
                act for act in db2 if name == act['name'] and rer_wo_string in act['location']
            ]
#             if len(act_rer_wo_xx) >= 1:
#                 print(act_rer_wo_xx)
            if len(act_rer_wo_xx) == 1:
                act_rer_wo_xx = act_rer_wo_xx[0]
                xx_location_long = act_rer_wo_xx['location'].replace(rer_wo_string, "")
                xx_location_iso2 = coco.convert(names=xx_location_long, to='ISO2')
                act_xx = [act for act in db2 if name == act['name'] and xx_location_iso2==act['location']]
                assert len(act_xx) == 1
                new_el[(name, loc)] = [act_rer_wo_xx, act_xx]
                mapping.append(new_el)
    return mapping

In [36]:
mapping_glo = create_location_mapping_glo(ag, ei_name)

In [37]:
mapping_rer = create_location_mapping_rer(ag, ei_name)

['reinforcing steel production' (kilogram, Europe without Austria, None)]
['nitric acid production, product in 50% solution state' (kilogram, RER w/o RU, None), 'nitric acid production, product in 50% solution state' (kilogram, RER w/o RU, None)]
['sawnwood production, softwood, raw, dried (u=20%)' (cubic meter, Europe without Switzerland, None)]


In [38]:
# Add nitric acid manually
nitric_acid_rer_wo_ru = [
    act for act in ei 
    if "nitric acid production" in act['name']
    and "RER w/o" in act['location']
    and "nitric acid, without water, in 50% solution state" in act['reference product']
]
assert len(nitric_acid_rer_wo_ru)==1
nitric_acid_ru = [
    act for act in ei 
    if "nitric acid production" in act['name']
    and "RU" == act['location']
    and "nitric acid, without water, in 50% solution state" in act['reference product']
]
assert len(nitric_acid_ru)==1
mapping_rer.append(
    {("nitric acid production, product in 50% solution state", "RER"): nitric_acid_rer_wo_ru + nitric_acid_ru}
)
mapping_rer

[{('reinforcing steel production',
   'RER'): ['reinforcing steel production' (kilogram, Europe without Austria, None), ['reinforcing steel production' (kilogram, AT, None)]]},
 {('sawnwood production, softwood, raw, dried (u=20%)',
   'RER'): ['sawnwood production, softwood, raw, dried (u=20%)' (cubic meter, Europe without Switzerland, None), ['sawnwood production, softwood, raw, dried (u=20%)' (cubic meter, CH, None)]]},
 {('nitric acid production, product in 50% solution state',
   'RER'): ['nitric acid production, product in 50% solution state' (kilogram, RER w/o RU, None), 'nitric acid production, product in 50% solution state' (kilogram, RU, None)]}]

In [39]:
mapping_glo

[{('market for transport, freight, lorry >32 metric ton, EURO4',
   'GLO'): ['market for transport, freight, lorry >32 metric ton, EURO4' (ton kilometer, RoW, None), 'market for transport, freight, lorry >32 metric ton, EURO4' (ton kilometer, RER, None)]},
 {('market for transport, freight, lorry, unspecified',
   'GLO'): ['market for transport, freight, lorry, unspecified' (ton kilometer, BR, None), 'market for transport, freight, lorry, unspecified' (ton kilometer, RER, None), 'market for transport, freight, lorry, unspecified' (ton kilometer, RoW, None), 'market for transport, freight, lorry, unspecified' (ton kilometer, ZA, None)]},
 {('market for lime', 'GLO'): ['market for lime' (kilogram, RER, None),
   'market for lime' (kilogram, RoW, None)]},
 {('market for iron sulfate',
   'GLO'): ['market for iron sulfate' (kilogram, RoW, None), 'market for iron sulfate' (kilogram, RER, None)]},
 {('market for transport, freight, inland waterways, barge',
   'GLO'): ['market for transport,